In [3]:
import numpy as np
import pandas as pd

# Revamping training data
It was discovered that the final submission has been bounded to be between 0 and 20, as such training data must also be bounded this

In [22]:
training_data = pd.read_csv('training_v6.csv')
training_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_prev_4_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth
0,0,0,2013,1,32,40,8.000000,0.000000,0.00,0.000000,6.0
1,0,0,2013,1,33,37,3.000000,0.000000,0.00,0.000000,3.0
2,0,0,2013,1,35,40,7.500000,0.000000,0.00,0.000000,1.0
3,0,0,2013,1,43,40,1.000000,0.000000,0.00,0.000000,1.0
4,0,0,2013,1,51,57,2.500000,0.000000,0.00,0.000000,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,2.75,119.000000,6.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,4.75,119.000000,2.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,0.75,59.666667,1.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,0.25,209.666667,1.0


In [12]:
# Given in that the expected output is bounded between 0 and 20 inclusive
training_data['item_cnt_mnth'].clip(upper=20,lower=0,inplace=True)
training_data['item_cnt_prev_3_mnth_mean'].clip(upper=20,lower=0,inplace=True)
training_data['item_cnt_mnthly_mean'].clip(upper=20,lower=0,inplace=True)
training_data['item_cnt_prev_4_mnth_mean'].clip(upper=20,lower=0,inplace=True)

In [23]:
train_grpd = training_data.groupby(['shop_id','item_id','date_block_num'])

# Time Lag
Adding time lag to number of items sold for the Nth previous month from the current

In [24]:
def add_prev_nth_mnth_sale(row, n, train):
    shop_id = row['shop_id']
    item_id = row['item_id']
    date_block_num = int(row['date_block_num'])
    prev_block = date_block_num - n
    key = (shop_id, item_id, prev_block)
    if prev_block < 0:
        return 0.0
    try:
        val = train.get_group(key)
        return float(val['item_cnt_mnth'])
    except KeyError as e:
        return 0.0

In [25]:
# We are going to add upto t-6 blocknums
for i in range(1,7):
    feat_name = 'item_cnt_mnth_t-'+str(i)
    training_data[feat_name] = training_data.apply(add_prev_nth_mnth_sale,
                                                  args=(i,train_grpd,),
                                                  axis=1)

In [26]:
training_data

,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_prev_4_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth,item_cnt_mnth_t-1,item_cnt_mnth_t-2,item_cnt_mnth_t-3,item_cnt_mnth_t-4,item_cnt_mnth_t-5,item_cnt_mnth_t-6
0,0,0,2013,1,32,40,8.000000,0.000000,0.00,0.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,2013,1,33,37,3.000000,0.000000,0.00,0.000000,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,2013,1,35,40,7.500000,0.000000,0.00,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,2013,1,43,40,1.000000,0.000000,0.00,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,2013,1,51,57,2.500000,0.000000,0.00,0.000000,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,59,33,2015,10,22087,83,6.424242,3.333333,2.75,119.000000,6.0,3.0,2.0,5.0,1.0,2.0,2.0
1609120,59,33,2015,10,22088,83,6.187500,5.000000,4.75,119.000000,2.0,1.0,7.0,7.0,4.0,3.0,3.0
1609121,59,33,2015,10,22091,83,2.294118,1.000000,0.75,59.666667,1.0,3.0,0.0,0.0,0.0,0.0,0.0
1609122,59,33,2015,10,22100,42,1.000000,0.333333,0.25,209.666667,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# reorder the training data
training_data = training_data[['item_cnt_mnth','shop_id','date_block_num','year','month','item_id',
                         'item_category_id','item_cnt_mnthly_mean','item_cnt_prev_3_mnth_mean',
                         'item_cnt_prev_4_mnth_mean','avg_item_price_3mnth_ago',
                         'item_cnt_mnth_t-1','item_cnt_mnth_t-2','item_cnt_mnth_t-3',
                         'item_cnt_mnth_t-4','item_cnt_mnth_t-5','item_cnt_mnth_t-6']]

In [28]:
training_data

,item_cnt_mnth,shop_id,date_block_num,year,month,item_id,item_category_id,item_cnt_mnthly_mean,item_cnt_prev_3_mnth_mean,item_cnt_prev_4_mnth_mean,avg_item_price_3mnth_ago,item_cnt_mnth_t-1,item_cnt_mnth_t-2,item_cnt_mnth_t-3,item_cnt_mnth_t-4,item_cnt_mnth_t-5,item_cnt_mnth_t-6
0,6.0,0,0,2013,1,32,40,8.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,0,0,2013,1,33,37,3.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0,0,2013,1,35,40,7.500000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0,0,2013,1,43,40,1.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,0,0,2013,1,51,57,2.500000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,6.0,59,33,2015,10,22087,83,6.424242,3.333333,2.75,119.000000,3.0,2.0,5.0,1.0,2.0,2.0
1609120,2.0,59,33,2015,10,22088,83,6.187500,5.000000,4.75,119.000000,1.0,7.0,7.0,4.0,3.0,3.0
1609121,1.0,59,33,2015,10,22091,83,2.294118,1.000000,0.75,59.666667,3.0,0.0,0.0,0.0,0.0,0.0
1609122,1.0,59,33,2015,10,22100,42,1.000000,0.333333,0.25,209.666667,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
training_data.to_csv('training_v7.csv',index=False)

0          6.0
1          3.0
2          1.0
3          1.0
4          2.0
          ... 
1609119    6.0
1609120    2.0
1609121    1.0
1609122    1.0
1609123    1.0
Name: item_cnt_mnth, Length: 1609124, dtype: float64